<a href="https://colab.research.google.com/github/hseongeun/projectmanager/blob/main/9_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 웹 스크래핑
(p.115)

In [ ]:
# requests 실습
import requests
url = 'https://ko.wikipedia.org/wiki/HTML'
res = requests.get(url)
res

<Response [200]>

### 태그 클래스

태그와 클래스를 한 번에 지정하려면 붙여서 표시

CSS 선택자 div.content (.을 class라고 생각하면 됨)

- <div>hello</div>
<span class="content">beautiful</div>
<div class="content text">world</div>

div만 하면 1,3번 선택 / content만 하면 2,3번 선택되므로 div.content하면 됨

In [ ]:
!pip install cssselect

In [ ]:
# 응답의 텍스트를 처리
import lxml.html
root = lxml.html.fromstring(res.text)

In [ ]:
# 개발자 도구(F12) 보면 title 태그 있는것 확인 가능
ts = root.cssselect('title')
# ts가 리스트기 때문에 첫번째 것을 추출해서 보여주라는 의미, 근데 확인해보면 여기는 어차피 하나만 들어있긴 함
ts[0].text_content()

'HTML - 위키백과, 우리 모두의 백과사전'

### 다음 뉴스 스크래핑

#### 기사 주소(링크) 수집

In [ ]:
# url에서 q : 검색어 p : 페이지
# q랑 p값만 한글/숫자로 바꾸면 검색 가능
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=인공지능&p={}'
res = requests.get(url.format(1))     # format(1) : p가 빈칸이니까 거기를 1로 채워라
root = lxml.html.fromstring(res.text)
for link in root.cssselect('span.cont_info a.f_nb'):
    text = link.text_content().strip()    # strip : 앞 뒤의 공백 없애줌
    if text == '다음뉴스':
        print(link.attrib['href'])    # 링크를 가져와라

http://v.daum.net/v/20230919132700610
http://v.daum.net/v/20230920180621489
http://v.daum.net/v/20230920190011686
http://v.daum.net/v/20230921114012945
http://v.daum.net/v/20230920225911543
http://v.daum.net/v/20230919120618559
http://v.daum.net/v/20230921060909465
http://v.daum.net/v/20230918090503295
http://v.daum.net/v/20230919132700610
http://v.daum.net/v/20230920180621489
http://v.daum.net/v/20230920190011686
http://v.daum.net/v/20230921114012945
http://v.daum.net/v/20230920225911543
http://v.daum.net/v/20230919120618559
http://v.daum.net/v/20230921060909465
http://v.daum.net/v/20230918090503295


In [ ]:
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=인공지능&p={}'

href = []
for page in range(1, 3):
    res = requests.get(url.format(page))
    root = lxml.html.fromstring(res.text)
    for link in root.cssselect('span.cont_info a.f_nb'):
        text = link.text_content().strip()
        if text == '다음뉴스':
            href.append(link.attrib['href'])
href

['http://v.daum.net/v/20230919132700610',
 'http://v.daum.net/v/20230920180621489',
 'http://v.daum.net/v/20230920190011686',
 'http://v.daum.net/v/20230921114012945',
 'http://v.daum.net/v/20230920225911543',
 'http://v.daum.net/v/20230919120618559',
 'http://v.daum.net/v/20230921114914326',
 'http://v.daum.net/v/20230921060909465',
 'http://v.daum.net/v/20230918090503295',
 'http://v.daum.net/v/20230921100311905',
 'http://v.daum.net/v/20230921090110814',
 'http://v.daum.net/v/20230916071016824',
 'http://v.daum.net/v/20230920174410592',
 'http://v.daum.net/v/20230921111116544',
 'http://v.daum.net/v/20230920170205211',
 'http://v.daum.net/v/20230920103232962',
 'http://v.daum.net/v/20230920090020899']

#### 기사 본문 스크랩

In [ ]:
# 기사 하나 긁어오기
res = requests.get('http://v.daum.net/v/20230919132700610')
root = lxml.html.fromstring(res.text)

# 기사 본문이 HTML의 class 태그의 article_view view에 있기 때문
article = root.cssselect('div.article_view')
article[0].text_content()


' \n         \n          \n           \n           \n            경기도가 챗지피티(ChatGPT) 같은 생성형 인공지능(AI)을 도정에 속속 도입하는 가운데 한계점 등을 두려워 말고 활용하기 시작해 위기를 기회로 만들어야 한다는 의견이 나왔다. 사진은 기업토론 모습. / 사진제공=경기도\n           \n          \n          \n           경기도가 챗지피티(ChatGPT) 같은 생성형 인공지능(AI)을 도정에 속속 도입하는 가운데 한계점 등을 두려워 말고 활용하기 시작해 위기를 기회로 만들어야 한다는 의견이 나왔다.\n          \n          경기도와 경기콘텐츠진흥원은 지난 14일부터 15일까지 수원컨벤션센터에서 \'인공지능이 확장한 문화기술의 세계\'를 주제로 \'2023년 문화기술 콘퍼런스\'를 열었다. 이번 행사는 챗지피티(ChatGPT) 같은 생성형 인공지능이 콘텐츠 산업계에 불러온 혁신을 조명하고 문화기술의 미래를 전망하기 위해 마련됐다.\n          이번 행사에는 \'인공지능의 미래\', \'인간은 필요없다\' 등을 저술한 제리 카플란(Jerry Kaplan) 교수와 유럽 인공지능 윤리학의 거두이자 \'특이점의 신화\'의 저자 장가브리엘 가나시아(Jean-Gabriel Ganascia) 소르본느 대학 교수 등 전 세계에서 저명한 석학들이 참석해 눈길을 끌었다.\n          장가브리엘 가나시아 소르본느 대학 교수는 경기도의 인공지능 도정 활용에 대해 "우선 기계를 사용해서 행정을 하고 있다는 것을 반드시 밝혀야 한다. 개인정보 침해를 조심해야 하고, 오답을 마치 정답처럼 말하는 환각(Hallucination) 문제를 경계해야 한다"며 "기계가 자동으로 무엇을 계속 생성하더라도 모든 것을 맡기지 말고, 대응 인력을 제대로 활용해야 한다"고 조언했다.\n          그러면서 "생성형 인공지능을 두려워하기보다 이용할 수 있어야 한다"며 "

In [ ]:
import tqdm.notebook
articles = []   # 기사 본문을 저장할 빈 리스트
for h in tqdm.notebook.tqdm(href):   # tqdm : 진행막대 표시
    res = requests.get(h)
    root = lxml.html.fromstring(res.text)
    for article in root.cssselect('div.article_view'):
        articles.append(article.text_content())
articles

#### 파일 저장

In [ ]:
import pandas as pd
news = pd.DataFrame({'주소': href, '본문': articles})
news.to_excel('기사.xlsx')

In [ ]:
# 기사 제목도 함께 긁어오기
import tqdm.notebook
titles = []
articles = []
for h in tqdm.notebook.tqdm(href):
    res = requests.get(h)
    root = lxml.html.fromstring(res.text)
    ta = zip(root.cssselect('h3.tit_view'),
             root.cssselect('div.article_view'))
    for title, article in ta:
        titles.append(title.text_content())
        articles.append(article.text_content())

  0%|          | 0/17 [00:00<?, ?it/s]

In [ ]:
# 기사 제목까지 함께 파일로 저장
import pandas as pd
news = pd.DataFrame({'주소': href, '제목': titles, '본문': articles})
news.to_excel('기사.xlsx')

In [ ]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 48.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.16.0-py3-none-any.whl size=30813255 sha256=a443c69171454132bd99fc22471599fa89f412ccf1bad4275d7a9d869948f4d2
  Stored in directory: /root/.cache/pip/wheels/6b/03/2b/ac2c97cc65ebd9df3516f4b900adc2f0a744df8d1375b2e2ef
Successfully built kiwipiepy-model


In [ ]:
# 1번 ppt p.276
from kiwipiepy import Kiwi
kiwi = Kiwi()
kiwi.add_user_word('인공지능', 'NNG')

True

In [ ]:
def extract_nouns(text):
    for token in kiwi.tokenize(text):
        if token.tag in {'NNG', 'NNP'}:
            yield token.form

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=100, tokenizer=extract_nouns)
dtm = cv.fit_transform(news['본문'])
word_count = pd.DataFrame({
    '단어': cv.get_feature_names_out(),
    '빈도': dtm.sum(axis=0).flat
})

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
word_count.sort_values('빈도', ascending=False).head(20)

## 텍스트 지도 학습

### 머신 러닝
**지도 학습(supervised learning)**

X -> Y관계의 데이터에서 X로 Y를 예측

X와 Y, 두 가지 데이터가 필요

보통 X가 텍스트

Y에 해당하는 데이터가 반드시 필요하나 구하기 어려울 수도 있음
- 예: 맛집 리뷰(X) 가 진짜(Y) 인가

**비지도 학습(unsupervised learning)**

데이터(X) 자체의 내재된 패턴을 발견
- 예: 유사한 고객 리뷰끼리 묶기(clustering)

지도학습과 달리 데이터에 Y가 없음

### 감성 분석 (지도 학습)

In [ ]:
import pandas as pd
df = pd.read_excel('yelp.xlsx')
df.head()

,review,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1000, stop_words='english')
dtm = cv.fit_transform(df.review)

In [ ]:
x = dtm
y = df.sentiment

#### 감성분석 실습 : 데이터 분할

데이터를 훈련(training) 데이터와 테스트(test) 데이터로 분할

훈련 데이터만을 모형에 적합 → 테스트 데이터로 평가
- 과소적합의 경우: 훈련 결과도, 테스트 결과도 나쁨
- 과대적합의 경우: 훈련 결과는 좋지만, 테스트 결과가 나쁨

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size=0.2, # 20%의 데이터를 테스트용으로 유보
    random_state=42) # 유사난수의 씨앗값 seed을 42로 설정

In [ ]:
y_train

29     0
535    1
695    0
557    0
836    1
      ..
106    1
270    1
860    1
435    0
102    1
Name: sentiment, Length: 800, dtype: int64

#### 정칙화(regularization)
과대적합을 막기 위한 방법들

w와 과대적합
- 기본적인 로지스틱 회귀분석은 교차 엔트로피를 가장 작게 만드는 파라미터 w를 추정
- w가 너무 커지면서 함수의 형태가 불규칙 → 과대적합

정칙화
- 손실함수 𝐽 (𝑤) 에 파라미터의 크기(norm)를 추가
- 가능한 작은 파라미터로 손실을 낮추도록 학습
- 함수형의 형태를 규칙적으로(=regular) 만들어 과대적합을 억제

종류
- 라쏘(lasso, L1) : 제약조건을 사각형으로 줌(p.163 참고)
- 릿지(ridge, L2) : 제약조건을 원형으로 줌
- 엘라스틱넷(elastic net) : 라쏘 + 릿지

#### 엘라스틱 넷

𝜆 : L1 L1과 L2 norm 중 L1의 반영 비율 (0<= 𝜆 <=1)

𝐶 : 정칙화의 강도를 결정

L1은 일부 파라미터를 0으로 만드는 경향이 있음

- 어떤 단어가 많고 적게 나오든지 긍/부정과는 무관

- 성능은 L2가 좋음

- L1과 L2의 비율을 정해야 함

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
model = LogisticRegressionCV(
    penalty='elasticnet', solver='saga', random_state=42,
    Cs=[0.1, 1, 10], l1_ratios=[0, 0.5, 1], max_iter=4000)
model.fit(x_train, y_train)

LogisticRegressionCV(Cs=[0.1, 1, 10], l1_ratios=[0, 0.5, 1], max_iter=4000,
                     penalty='elasticnet', random_state=42, solver='saga')

In [ ]:
# 1일때가 가장 좋더라
model.C_

array([1.])

In [ ]:
# L1_ratio가 0일 때 가장 적합하다 = 원일 때 적합하다
model.l1_ratio_   # ratio : 비율

array([0])

In [ ]:
model.score(x_train, y_train)

0.94125

In [ ]:
model.score(x_test, y_test)

0.76

In [ ]:
# 단어별 가중치
word_coef = pd.DataFrame({
    '단어': cv.get_feature_names_out(),
    '가중치': model.coef_.flat
})

In [ ]:
# 긍정 단어
word_coef.sort_values('가중치').tail(10)

,단어,가중치
208,excellent,1.340621
221,fantastic,1.439516
250,friendly,1.524526
364,love,1.525609
265,good,1.532660
409,nice,1.555230
32,awesome,1.714438
12,amazing,1.779166
153,delicious,2.201475
268,great,2.867876


In [ ]:
# 부정 단어
word_coef.sort_values('가중치').head(10)

,단어,가중치
37,bad,-1.448292
179,don,-1.397359
65,bland,-1.274375
939,wasn,-1.225014
392,minutes,-1.177965
977,worst,-1.146635
607,rude,-1.040134
889,unfortunately,-1.039067
703,slow,-1.036536
546,probably,-1.015125
